In [28]:
from flask import Flask, render_template_string, request, jsonify
import pandas as pd
import numpy as np
import joblib
from tensorflow.keras.models import load_model
import os

# Create Flask app
app = Flask(__name__)

def load_model_and_transformers():
    """Load the trained model and preprocessing transformers."""
    try:
        # Try loading Neural Network model (.h5)
        model = load_model('../models/heart_disease_model.h5')
        is_neural_network = True
    except:
        # If not found, try loading other model (.pkl)
        model = joblib.load('../models/heart_disease_model.pkl')
        is_neural_network = False
    
    scaler = joblib.load('../models/scaler.pkl')
    label_encoders = joblib.load('../models/label_encoders.pkl')
    return model, scaler, label_encoders, is_neural_network

def preprocess_input(input_data, label_encoders, scaler):
    """Preprocess the input data for prediction."""
    df = pd.DataFrame([input_data])
    
    # Encode categorical variables
    categorical_cols = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
    for col in categorical_cols:
        df[col] = label_encoders[col].transform(df[col])
    
    # Scale features
    X = scaler.transform(df)
    return X

def make_prediction(model, X, is_neural_network):
    """Make prediction using the appropriate model type."""
    if is_neural_network:
        prediction = model.predict(X)[0][0]
    else:
        if hasattr(model, 'predict_proba'):
            prediction = model.predict_proba(X)[0][1]
        else:
            prediction = model.predict(X)[0]
    return prediction

# HTML template
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Heart Disease Prediction</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        .card {
            border-radius: 15px;
            border: none;
        }
        .card-body {
            padding: 2rem;
        }
        .form-label {
            font-weight: 500;
            color: #2c3e50;
        }
        .form-control, .form-select {
            border-radius: 8px;
            padding: 0.75rem;
            border: 1px solid #e0e0e0;
            transition: all 0.3s ease;
        }
        .btn-primary {
            background-color: #4a90e2;
            border: none;
            padding: 0.75rem 2rem;
            border-radius: 8px;
            transition: all 0.3s ease;
        }
        .progress {
            border-radius: 10px;
            background-color: #f5f5f5;
            margin-top: 1rem;
        }
    </style>
</head>
<body class="bg-light">
    <div class="container py-5">
        <h1 class="text-center mb-4">Heart Disease Prediction</h1>
        <div class="row justify-content-center">
            <div class="col-md-10">
                <div class="card shadow">
                    <div class="card-body">
                        <form id="prediction-form" class="needs-validation" novalidate>
                            <div class="row">
                                <!-- Left Column -->
                                <div class="col-md-6">
                                    <div class="mb-3">
                                        <label for="age" class="form-label">Age</label>
                                        <input type="number" class="form-control" id="age" name="age" min="20" max="100" value="40" required>
                                    </div>
                                    <div class="mb-3">
                                        <label for="sex" class="form-label">Sex</label>
                                        <select class="form-select" id="sex" name="sex" required>
                                            <option value="M">Male</option>
                                            <option value="F">Female</option>
                                        </select>
                                    </div>
                                    <div class="mb-3">
                                        <label for="chest_pain" class="form-label">Chest Pain Type</label>
                                        <select class="form-select" id="chest_pain" name="chest_pain" required>
                                            <option value="ATA">ATA</option>
                                            <option value="NAP">NAP</option>
                                            <option value="ASY">ASY</option>
                                            <option value="TA">TA</option>
                                        </select>
                                    </div>
                                    <div class="mb-3">
                                        <label for="resting_bp" class="form-label">Resting Blood Pressure</label>
                                        <input type="number" class="form-control" id="resting_bp" name="resting_bp" min="80" max="200" value="120" required>
                                    </div>
                                    <div class="mb-3">
                                        <label for="cholesterol" class="form-label">Cholesterol</label>
                                        <input type="number" class="form-control" id="cholesterol" name="cholesterol" min="100" max="600" value="200" required>
                                    </div>
                                </div>
                                <!-- Right Column -->
                                <div class="col-md-6">
                                    <div class="mb-3">
                                        <label for="fasting_bs" class="form-label">Fasting Blood Sugar > 120 mg/dl</label>
                                        <select class="form-select" id="fasting_bs" name="fasting_bs" required>
                                            <option value="0">No</option>
                                            <option value="1">Yes</option>
                                        </select>
                                    </div>
                                    <div class="mb-3">
                                        <label for="resting_ecg" class="form-label">Resting ECG</label>
                                        <select class="form-select" id="resting_ecg" name="resting_ecg" required>
                                            <option value="Normal">Normal</option>
                                            <option value="ST">ST</option>
                                            <option value="LVH">LVH</option>
                                        </select>
                                    </div>
                                    <div class="mb-3">
                                        <label for="max_hr" class="form-label">Maximum Heart Rate</label>
                                        <input type="number" class="form-control" id="max_hr" name="max_hr" min="60" max="220" value="150" required>
                                    </div>
                                    <div class="mb-3">
                                        <label for="exercise_angina" class="form-label">Exercise-Induced Angina</label>
                                        <select class="form-select" id="exercise_angina" name="exercise_angina" required>
                                            <option value="Y">Yes</option>
                                            <option value="N">No</option>
                                        </select>
                                    </div>
                                    <div class="mb-3">
                                        <label for="oldpeak" class="form-label">ST Depression (Oldpeak)</label>
                                        <input type="number" class="form-control" id="oldpeak" name="oldpeak" min="0" max="6" step="0.1" value="0.0" required>
                                    </div>
                                    <div class="mb-3">
                                        <label for="st_slope" class="form-label">ST Slope</label>
                                        <select class="form-select" id="st_slope" name="st_slope" required>
                                            <option value="Up">Up</option>
                                            <option value="Flat">Flat</option>
                                            <option value="Down">Down</option>
                                        </select>
                                    </div>
                                </div>
                            </div>
                            <div class="text-center mt-4">
                                <button type="submit" class="btn btn-primary btn-lg">Predict</button>
                            </div>
                        </form>

                        <!-- Results Section -->
                        <div id="results" class="mt-4" style="display: none;">
                            <hr>
                            <h3 class="text-center mb-4">Prediction Results</h3>
                            <div class="alert" role="alert" id="result-message"></div>
                            <div class="progress" style="height: 30px;">
                                <div class="progress-bar" role="progressbar" id="risk-bar" 
                                     aria-valuenow="0" aria-valuemin="0" aria-valuemax="100"></div>
                            </div>
                        </div>
                    </div>
                </div>
            </div>
        </div>
    </div>

    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
    <script>
        document.addEventListener('DOMContentLoaded', function() {
            const form = document.getElementById('prediction-form');
            const resultsDiv = document.getElementById('results');
            const resultMessage = document.getElementById('result-message');
            const riskBar = document.getElementById('risk-bar');

            form.addEventListener('submit', async function(e) {
                e.preventDefault();
                
                // Show loading state
                const submitButton = form.querySelector('button[type="submit"]');
                const originalButtonText = submitButton.innerHTML;
                submitButton.innerHTML = '<span class="spinner-border spinner-border-sm" role="status" aria-hidden="true"></span> Predicting...';
                submitButton.disabled = true;

                try {
                    // Create FormData object
                    const formData = new FormData(form);
                    
                    // Send POST request to Flask backend
                    const response = await fetch('/predict', {
                        method: 'POST',
                        body: formData
                    });
                    
                    const data = await response.json();
                    
                    if (response.ok) {
                        // Show results
                        resultsDiv.style.display = 'block';
                        
                        // Update message and alert type
                        resultMessage.className = 'alert ' + (data.risk_level === 'High' ? 'alert-danger' : 'alert-success');
                        resultMessage.textContent = data.message;
                        
                        // Update progress bar
                        riskBar.style.width = data.probability + '%';
                        riskBar.setAttribute('aria-valuenow', data.probability);
                        
                        // Update progress bar color based on risk level
                        if (data.probability <= 30) {
                            riskBar.className = 'progress-bar bg-success';
                        } else if (data.probability <= 70) {
                            riskBar.className = 'progress-bar bg-warning';
                        } else {
                            riskBar.className = 'progress-bar bg-danger';
                        }
                        
                        // Scroll to results
                        resultsDiv.scrollIntoView({ behavior: 'smooth' });
                    } else {
                        throw new Error(data.error || 'Prediction failed');
                    }
                } catch (error) {
                    // Show error message
                    resultsDiv.style.display = 'block';
                    resultMessage.className = 'alert alert-danger';
                    resultMessage.textContent = 'Error: ' + error.message;
                } finally {
                    // Restore button state
                    submitButton.innerHTML = originalButtonText;
                    submitButton.disabled = false;
                }
            });

            // Form validation
            form.querySelectorAll('input, select').forEach(input => {
                input.addEventListener('input', function() {
                    if (this.checkValidity()) {
                        this.classList.remove('is-invalid');
                        this.classList.add('is-valid');
                    } else {
                        this.classList.remove('is-valid');
                        this.classList.add('is-invalid');
                    }
                });
            });
        });
    </script>
</body>
</html>
"""

@app.route('/')
def home():
    """Render the home page."""
    return render_template_string(html_template)

@app.route('/predict', methods=['POST'])
def predict():
    """Handle prediction requests."""
    try:
        # Load model and transformers
        model, scaler, label_encoders, is_neural_network = load_model_and_transformers()
        
        # Get input data from form
        input_data = {
            'Age': int(request.form['age']),
            'Sex': request.form['sex'],
            'ChestPainType': request.form['chest_pain'],
            'RestingBP': int(request.form['resting_bp']),
            'Cholesterol': int(request.form['cholesterol']),
            'FastingBS': int(request.form['fasting_bs']),
            'RestingECG': request.form['resting_ecg'],
            'MaxHR': int(request.form['max_hr']),
            'ExerciseAngina': request.form['exercise_angina'],
            'Oldpeak': float(request.form['oldpeak']),
            'ST_Slope': request.form['st_slope']
        }
        
        # Preprocess input
        X = preprocess_input(input_data, label_encoders, scaler)
        
        # Make prediction
        prediction = make_prediction(model, X, is_neural_network)
        probability = prediction * 100
        
        # Prepare response
        result = {
            'probability': float(probability),
            'risk_level': 'High' if probability > 50 else 'Low',
            'message': f'Warning: High risk of heart disease ({probability:.1f}% probability)' 
                      if probability > 50 
                      else f'Low risk of heart disease ({probability:.1f}% probability)'
        }
        
        return jsonify(result)
        
    except Exception as e:
        return jsonify({'error': str(e)}), 400

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Feb/2025 16:32:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2025 16:32:07] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2025 16:32:47] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Feb/2025 16:33:18] "POST /predict HTTP/1.1" 200 -
